In [79]:
import numpy as np
import pickle
import numpy as np
from sklearn import metrics
from sklearn.preprocessing import label_binarize
from sklearn.ensemble import RandomForestClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multiclass import OneVsOneClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import numpy as np
from sklearn import metrics
from matplotlib import pyplot as plt
from sklearn.utils import shuffle
%matplotlib inline

In [80]:
embedding=np.loadtxt("wiki_b_comp/embedding.txt")


In [81]:
itw=pickle.load(open('wiki_b_comp/itw.p','rb'))
wti=pickle.load(open('wiki_b_comp/wti.p','rb'))

In [88]:
len(embedding)

1307082

In [83]:
b_i=np.loadtxt("wiki_b_comp/bias_i.txt")

In [84]:
embedding=np.concatenate((embedding,b_i.reshape(-1,1)),axis=1)

In [85]:
wto=pickle.load(open('wiki_b_comp/wto.p','rb'))

In [86]:
# fpeople=pickle.load(open('train_new/Forbes_People_Final.pickle','rb'))
ncomp=pickle.load(open('train_new/Nasdaq_Company_Final_revised','rb'))
scomp=pickle.load(open('train_new/Sec_Company_Final_revised','rb'))
# speople=pickle.load(open('train_new/Sec_People_Final.pickle','rb'))


In [89]:
# fpeople=fpeople.rename(index=str, columns={"People": "entity"})
# speople=speople.rename(index=str, columns={"People": "entity"})
ncomp=ncomp.rename(index=str,columns={"Company":"entity"})
scomp=scomp.rename(index=str,columns={"Company":"entity"})

In [90]:
big_df=ncomp.append(scomp)

In [91]:
index=[]
raw_name=[]
new_name=[]
poc=[]
embeds=[]
target=[]
showup=[]
bias=[]
for k,v in itw.items():
    index.append(k)
    raw_name.append(v)
    showup.append(wto[v])
    new_name.append(v[:-2])
    bias.append(b_i[k])
    if v[-1]=='1':
        poc.append('p')
    else:
        poc.append('c')
    embeds.append(embedding[k,:])
    target.append(' ')

In [92]:
import pandas as pd

weknow=pd.DataFrame({'index':index,'raw_name':raw_name,'new_name':new_name,'poc':poc,'feature':embeds,'showup':showup,'target':target,'bias':bias})

In [93]:
from collections import Counter
cnt = Counter()
temp=[]
for w in big_df.entity:
    temp.append(w)
    #temp.append(w)
cnt.update(temp)

In [94]:
weknow=weknow[weknow.new_name.isin(cnt)]

In [95]:
weknow=weknow.join(big_df.set_index('entity'), on='new_name',how='inner')

In [96]:
weknow=(weknow.drop_duplicates(subset=weknow.columns[2:-1]))

In [97]:
weknow=weknow.sort_values(by=['showup'],ascending=False)

In [98]:
weknow.groupby(['Industry']).count()

,bias,feature,index,new_name,poc,raw_name,showup,target
Industry,,,,,,,,
Basic Industries,639,639,639,639,639,639,639,639
Consumer Services,1467,1467,1467,1467,1467,1467,1467,1467
Finance,784,784,784,784,784,784,784,784
Health Care,481,481,481,481,481,481,481,481
Media,97,97,97,97,97,97,97,97
Miscellaneous,938,938,938,938,938,938,938,938
Public Utilities,370,370,370,370,370,370,370,370
Technology,957,957,957,957,957,957,957,957
Transportation,292,292,292,292,292,292,292,292


In [19]:
len(weknow)

6025

In [99]:
good_df=weknow

In [101]:
good_df=pd.DataFrame()
for ind in np.unique(weknow.Industry):
    ind_df=weknow[weknow.Industry==ind]
    origi_len=len(ind_df)
    
    #ind_df_high=ind_df.iloc[:int(origi_len*0.2)].copy()
    #ind_df_high=ind_df[ind_df.showup>1000].copy()
    ind_df_low=ind_df.iloc[:int(origi_len*0.5)].copy()
    
    #ind_df=pd.concat((ind_df_high,ind_df_low),axis=0)
    ind_df=ind_df_low
    
    good_df=pd.concat((good_df,ind_df),axis=0)

In [102]:
good_df=good_df.sort_values(by=['showup'],ascending=False)

In [103]:
good_df.groupby(['Industry']).count()

,bias,feature,index,new_name,poc,raw_name,showup,target
Industry,,,,,,,,
Basic Industries,319,319,319,319,319,319,319,319
Consumer Services,733,733,733,733,733,733,733,733
Finance,392,392,392,392,392,392,392,392
Health Care,240,240,240,240,240,240,240,240
Media,48,48,48,48,48,48,48,48
Miscellaneous,469,469,469,469,469,469,469,469
Public Utilities,185,185,185,185,185,185,185,185
Technology,478,478,478,478,478,478,478,478
Transportation,146,146,146,146,146,146,146,146


In [65]:
#good_df=shuffle(good_df)
good_df

,feature,index,new_name,poc,raw_name,showup,target,Industry
29,"[1.05556869507, -2.48249769211, -0.46173843741...",29,united,c,united_2,348977,,Finance
5455,"[0.462655156851, -0.0800175741315, -0.06480161...",5455,cbs,c,cbs_2,86582,,Technology
272,"[-1.74163806438, 0.15512534976, -0.56323456764...",272,microsoft,c,microsoft_2,53821,,Technology
4411,"[1.89409506321, -0.279261380434, 0.66251003742...",4411,manchester united,c,manchester united_2,40727,,Consumer Services
1001,"[0.229396700859, 0.154855400324, 0.06758229434...",1001,continental,c,continental_2,36544,,Finance
5634,"[-0.413626432419, -0.586589276791, -0.00060242...",5634,union,c,union_2,30606,,Consumer Services
2205,"[0.670178472996, -0.673588812351, 0.5095498561...",2205,sony,c,sony_2,29295,,Technology
198,"[0.410659432411, -0.807502388954, -1.102667570...",198,google,c,google_2,27254,,Consumer Services
16318,"[1.57156658173, -0.3465321064, -0.530379831791...",16318,lockheed,c,lockheed_2,23055,,Transportation
251,"[0.45208260417, -0.52452403307, 0.901065826416...",251,manchester,c,manchester_2,20253,,Miscellaneous


# Make split

In [63]:
# train_len=3500


# test_df=df[train_len:]
# df=df[:train_len]

df=good_df.copy()
df=df[df.Industry != 'Media']
df=df[df.Industry != 'Public Utilities']
df=df[df.Industry != 'Transportation']
df=df[df.Industry != 'Health Care']

min_count=df.groupby('Industry').count().min()[0]
traindf=pd.DataFrame()
testdf=pd.DataFrame()
for ind in np.unique(df['Industry']):
    print(ind)
    count=len(df[df.Industry==ind])
    #print(count)
    #if count>min_count:
    new_index=(list(np.random.choice(df[df.Industry==ind].index,min_count,replace=False)))
    print(len(df.loc[new_index]))
    
    traindf=pd.concat((traindf,df.loc[new_index[:int(len(new_index)*0.8)]]),axis=0)
    testdf=pd.concat((testdf,df.loc[new_index[int(len(new_index)*0.8):]]),axis=0)

old_index=np.arange(len(traindf))
print(len(old_index))
np.random.shuffle(old_index)
print(len(old_index))
traindf=traindf.iloc[old_index]
print(len(traindf))
    
old_index=np.arange(len(testdf))
print(len(old_index))
np.random.shuffle(old_index)
print(len(old_index))
testdf=testdf.iloc[old_index]
print(len(traindf))


train_x=[]
for vec in (traindf.feature):
    train_x.append(list(vec))
test_x=[]
for vec in (testdf.feature):
    test_x.append(list(vec))
train_x=np.array(train_x)
test_x=np.array(test_x)

train_y=traindf.Industry
test_y=testdf.Industry

Basic Industries
447
Consumer Services
447
Finance
447
Miscellaneous
447
Technology
447
1785
1785
1785
450
450
1785


In [64]:
print(test_x.shape)
print(test_y.shape)
print(train_x.shape)
print(train_y.shape)

(450, 150)
(450,)
(1785, 150)
(1785,)


In [65]:
ind_list=np.unique(df['Industry'])
ind_list

array(['Basic Industries', 'Consumer Services', 'Finance', 'Miscellaneous',
       'Technology'], dtype=object)

# Multi_class

In [43]:
o_good_df=good_df
good_df=good_df[good_df.Industry!='Media']
good_df=good_df[good_df.Industry!='Transportation']
#good_df=good_df[good_df.Industry.isin(['Miscellaneous','Consumer Services','Finance'])]

In [44]:
ind_list=np.unique(good_df['Industry'])
itd={}

In [45]:
ind_list

array(['Basic Industries', 'Consumer Services', 'Finance', 'Health Care',
       'Miscellaneous', 'Public Utilities', 'Technology'], dtype=object)

In [46]:
# df=good_df

train_class=[]
test_class=[]

big_test_x=[]
big_train_x=[]
clfs=[]
n=0
deltas=[]

new_test=pd.DataFrame()
new_df=pd.DataFrame()

for ind in ind_list:
    
    sub_ind=good_df[good_df.Industry==ind].copy()
    #test_index=sub_ind.index[:int(len(sub_ind)*0.2)]
    test_index=sub_ind.index[int(len(sub_ind)*0.3):int(len(sub_ind)*0.5)]
    new_test=pd.concat((new_test,sub_ind.loc[test_index].copy()),axis=0)
    
    new_df=pd.concat((new_df,sub_ind.drop(test_index).copy()),axis=0)
    

In [47]:
new_df=new_df.sort_values(by=['showup'],ascending=False)
new_test=new_test.sort_values(by=['showup'],ascending=False)

In [70]:
for ind in ind_list:
    itd[ind]=n
    df=new_df.copy()
    df['istech']=(df['Industry']==ind)

    tech_df=df[df.istech]
    non_tech_df=df[-df.istech]

    train_x=[]
    train_y=[]
    
    train_len=len(tech_df)
    end_len=len(tech_df)
    total=len(tech_df)
    
    
    train_class+=[n]*train_len
    
    for vec in (tech_df.feature):
        train_x.append(list(vec))

    for vec in (non_tech_df.iloc[:train_len].feature):
        train_x.append(list(vec))
        
    train_y=np.array([1]*train_len+[0]*train_len)


    print(np.unique(non_tech_df.iloc[:train_len].Industry))
    clf=LogisticRegression()
    #clf =RandomForestClassifier(n_estimators=1000,max_depth=10,criterion='entropy',n_jobs=8,max_features=50)
    #clf=SVC(kernel='poly',probability=True,degree=2,C=1e6,tol=1e-06)
    clf.fit(train_x,train_y)
    clfs.append(clf)
    
    
    n+=1
    
    
    pred_prob=clf.predict_proba(train_x)
    fpr, tpr, thresholds = metrics.roc_curve(train_y, pred_prob[:,1], pos_label=1)
    auc=metrics.auc(fpr, tpr)
    accuracy=sum(np.argmax(pred_prob,1)==train_y)/len(train_y)

    print("rf auc: ",auc)
    print("rf accuracy: ",accuracy)
    
    best=0
    dd=0
    acc=0
    
    dd=np.mean(pred_prob[train_y==1,1])
    
#     for delta in np.arange(0,0.9,0.01):
#         mody_pred=pred_prob.copy()
#         mody_pred[:,0]=mody_pred[:,0]+delta
#         f_pred=np.argmax(mody_pred,1)
#         if sum(f_pred==train_y)/len(f_pred)>= best:
#             best=sum(f_pred==train_y)/len(f_pred)
#             dd=delta
#             recall=sum(f_pred[f_pred==train_y]==1)/sum(train_y==1)
    deltas.append(dd)
#     print('best accu ',best)
    print('delta ',dd)
#     print('recall',recall)
    

['Consumer Services' 'Finance' 'Health Care' 'Miscellaneous'
 'Public Utilities' 'Technology']
rf auc:  0.962948211749
rf accuracy:  0.917808219178
delta  0.853792999476
['Basic Industries' 'Finance' 'Health Care' 'Miscellaneous'
 'Public Utilities' 'Technology']
rf auc:  0.811871497436
rf accuracy:  0.732964224872
delta  0.628865116709
['Basic Industries' 'Consumer Services' 'Health Care' 'Miscellaneous'
 'Public Utilities' 'Technology']
rf auc:  0.933314509995
rf accuracy:  0.872408293461
delta  0.791833790893
['Basic Industries' 'Consumer Services' 'Finance' 'Miscellaneous'
 'Public Utilities' 'Technology']
rf auc:  0.99812447293
rf accuracy:  0.992207792208
delta  0.960986718125
['Basic Industries' 'Consumer Services' 'Finance' 'Health Care'
 'Public Utilities' 'Technology']
rf auc:  0.921413333333
rf accuracy:  0.858666666667
delta  0.767198568501
['Basic Industries' 'Consumer Services' 'Finance' 'Health Care'
 'Miscellaneous' 'Technology']
rf auc:  0.999155405405
rf accuracy:  0.

In [71]:
big_test_x=[]
for vec in new_test.feature:
    big_test_x.append(vec)
big_test_x=np.array(big_test_x)
n=0
test_class=[]
for ind in new_test.Industry:
    test_class.append(np.where(ind_list == ind)[0][0])

In [66]:
n=0
test_class=np.array(test_class)
for clf in clfs:
    tt=big_test_x[test_class==n,:]
    ttn=big_test_x[test_class!=n,:]
    #np.random.shuffle(ttn)
    ttn=ttn[:len(tt),:]
    #ttn=big_test_x[test_class!=n,:]
    
    ttt=np.concatenate((tt,ttn),axis=0)
    yy=len(tt)*[1]+len(ttn)*[0]
    ppp=clf.predict_proba(ttt)
    print('tt len',len(tt))
    print('yy len',len(yy))
    pred_y=np.argmax(ppp, axis=1)
    #test=np.argmax(test_class, axis=1)
    accu=sum(pred_y==np.array(yy))/len(yy)
    print('acc :     ',accu)
    
    fpr, tpr, thresholds = metrics.roc_curve(yy, ppp[:,1], pos_label=1)
    print('auc', metrics.auc(fpr, tpr))
    
    n+=1

tt len 45
yy len 90
acc :      0.5
auc 0.875308641975
tt len 103
yy len 206
acc :      0.592233009709
auc 0.881515694222
tt len 55
yy len 110
acc :      0.527272727273
auc 0.888429752066
tt len 34
yy len 68
acc :      0.529411764706
auc 1.0
tt len 66
yy len 132
acc :      0.537878787879
auc 0.952708907254
tt len 26
yy len 52
acc :      0.5
auc 0.806213017751
tt len 67
yy len 134
acc :      0.55223880597
auc 0.673646691914
tt len 21
yy len 42
acc :      0.5
auc 0.235827664399


In [56]:
ppp

array([[ 0.29 ,  0.71 ],
       [ 0.318,  0.682],
       [ 0.223,  0.777],
       [ 0.372,  0.628],
       [ 0.19 ,  0.81 ],
       [ 0.189,  0.811],
       [ 0.199,  0.801],
       [ 0.177,  0.823],
       [ 0.191,  0.809],
       [ 0.283,  0.717],
       [ 0.287,  0.713],
       [ 0.32 ,  0.68 ],
       [ 0.088,  0.912],
       [ 0.234,  0.766],
       [ 0.196,  0.804],
       [ 0.079,  0.921],
       [ 0.27 ,  0.73 ],
       [ 0.197,  0.803],
       [ 0.238,  0.762],
       [ 0.093,  0.907],
       [ 0.075,  0.925],
       [ 0.146,  0.854],
       [ 0.08 ,  0.92 ],
       [ 0.274,  0.726],
       [ 0.268,  0.732],
       [ 0.089,  0.911],
       [ 0.138,  0.862],
       [ 0.091,  0.909],
       [ 0.059,  0.941],
       [ 0.155,  0.845],
       [ 0.194,  0.806],
       [ 0.22 ,  0.78 ],
       [ 0.145,  0.855],
       [ 0.05 ,  0.95 ],
       [ 0.08 ,  0.92 ],
       [ 0.302,  0.698],
       [ 0.134,  0.866],
       [ 0.06 ,  0.94 ],
       [ 0.074,  0.926],
       [ 0.047,  0.953],


In [77]:
pred_prob_train=[]
pred_prob_test=[]
n=0
for clf in clfs:
#     this_pred=clf.predict_proba(big_train_x)[:,0].reshape(-1,1)+deltas[n]
#     if len(pred_prob_train)==0:
#         pred_prob_train=this_pred
#     else:
#         pred_prob_train=np.concatenate((pred_prob_train,this_pred),axis=1)
    
    
    this_pred=clf.predict_proba(big_test_x)[:,0].reshape(-1,1)
    #this_pred=-this_pred

    if len(pred_prob_test)==0:
        pred_prob_test=this_pred
    else:
        pred_prob_test=np.concatenate((pred_prob_test,this_pred),axis=1)
    n+=1

In [78]:
pred_y=np.argmax((pred_prob_test), axis=1)
#test=np.argmax(test_class, axis=1)
accu=sum(pred_y==np.array(test_class))/len(test_class)
print(accu)

0.123336291038


In [59]:
svm:0.2325
rf: 0.1881
lg: 0.1233

array([[  1.18361370e-02,   5.62985117e-01,   1.63409292e-01,
          1.62935474e-02,   1.62175517e-01,   2.82679162e-02,
          1.08764623e-01],
       [  5.58792294e-02,   2.85813696e-01,   7.52736549e-02,
          1.32771914e-05,   2.19305432e-01,   3.89744926e-03,
          4.54036126e-01],
       [  4.66054636e-01,   2.57004157e-01,   4.69568827e-01,
          8.94436649e-08,   4.46298730e-01,   1.06248091e-06,
          7.87658445e-01],
       [  2.69401131e-02,   3.13488841e-01,   2.66419790e-01,
          3.80563164e-03,   1.52619291e-01,   6.55696248e-03,
          2.20082586e-01],
       [  7.07117019e-02,   3.71342035e-01,   7.85373509e-02,
          5.51843049e-08,   1.53505571e-01,   4.37353508e-03,
          1.04380771e-01],
       [  1.66273386e-01,   6.14295210e-01,   1.92370855e-01,
          3.56639461e-06,   3.21566509e-01,   6.18063405e-03,
          4.29959431e-01],
       [  7.60842509e-03,   3.99297183e-01,   1.95628043e-01,
          5.65372975e-03,   1.90

In [ ]:
svm 0.2165

In [42]:
ind_list

array(['Basic Industries', 'Consumer Services', 'Finance', 'Health Care',
       'Media', 'Miscellaneous', 'Public Utilities', 'Technology',
       'Transportation'], dtype=object)

In [50]:
for i in range(len(pred_prob_test[0,:])):
    p=pred_prob_test[:,i]
    t=np.array(test_class)
    print(sum(t==i))
#     y = test_y+1
#     pred = lg.predict_proba(test_x)[:,1]
    fpr, tpr, thresholds = metrics.roc_curve(t, p, pos_label=i)
    print(metrics.auc(fpr, tpr))

45
0.54743130227
103
0.382011007359
55
0.564741336012
34
0.0799032406696
66
0.387917637918
26
0.650452488688
67
0.70933901919
21
0.940235690236


# Binary

In [107]:
for ind in ind_list:
    df=good_df.copy()

    #df=shuffle(df)
    print("-"*50)
    print("Industry: ", ind)
    df['istech']=(df['Industry']==ind)

    tech_df=df[df.istech].copy()
    non_tech_df=df[-df.istech].copy()
    
    
    
    train_x=[]
    train_y=[]

    #tech_train_index=np.random.choice(tech_df.index,int(len(tech_df.index)*0.8),replace=False)
    tech_train_index=tech_df.index[:int(len(tech_df.index)*0.8)]
    
    tech_test_index=tech_df.drop(tech_train_index).index
    
    #nontech_train_index=np.random.choice(non_tech_df.index,len(tech_train_index),replace=False)
    nontech_train_index=non_tech_df.index[:len(tech_train_index)]
    nontech_test_index=non_tech_df.index[len(tech_train_index):len(tech_train_index)+len(tech_test_index)]
    
    
#     print(len(tech_train_index))
#     print(len(tech_test_index))
    
#     print(len(nontech_train_index))
#     print(len(nontech_test_index))
    
    for vec in (tech_df.loc[tech_train_index].feature):
        train_x.append(list(vec))
    for vec in (non_tech_df.loc[nontech_train_index].feature):
        train_x.append(list(vec))
    train_y=np.array([1]*len(tech_train_index)+[0]*len(nontech_train_index))
#     test_x=[]
#     for vec in (tech_df.iloc[train_len:total].feature):
#         test_x.append(list(vec))
#     for vec in (non_tech_df.iloc[train_len:total].feature):
#         test_x.append(list(vec))
#     test_y=np.array([1]*len(tech_df.iloc[train_len:total].feature)+[0]*len(non_tech_df.iloc[train_len:total]))

    test_x=[]
    for vec in (tech_df.loc[tech_test_index].feature):
        test_x.append(list(vec))
        
#     s_df=non_tech_df.iloc[train_len:]
#     s_df=shuffle(s_df)
#     for vec in (s_df.iloc[:total-train_len].feature):
#         test_x.append(list(vec))
        
    for vec in (non_tech_df.loc[nontech_test_index].feature):
        test_x.append(list(vec))
    test_y=np.array([1]*len(tech_test_index)+[0]*len(nontech_test_index))

    
    train_x=np.array(train_x)
    test_x=np.array(test_x)

    clf = RandomForestClassifier(n_estimators=1000,max_depth=10,criterion='entropy',n_jobs=8,max_features=50)
    clf.fit(train_x,train_y)

    D=np.mean(clf.predict_proba(train_x[train_y==1])[:,1])-np.mean(clf.predict_proba(train_x[train_y==1])[:,0])
    print('D',D)
    pred_prob=clf.predict_proba(test_x)
    fpr, tpr, thresholds = metrics.roc_curve(test_y, pred_prob[:,1], pos_label=1)
    auc=metrics.auc(fpr, tpr)
    pred_prob[:,0]=pred_prob[:,0]+D
    accuracy=sum(np.argmax(pred_prob,1)==test_y)/len(test_y)

    print("rf auc: ",auc)
    print("rf accuracy: ",accuracy)
    

#     best=0
#     dd=0
#     acc=0
#     for delta in np.arange(0,0.9,0.01):
#         mody_pred=pred_prob.copy()
#         mody_pred[:,0]=mody_pred[:,0]+delta
#         f_pred=np.argmax(mody_pred,1)
#         if sum(f_pred[f_pred==test_y]==1)/sum(f_pred==1)> best:
#             best=sum(f_pred[f_pred==test_y]==1)/sum(f_pred==1)
#             dd=delta
#             recall=sum(f_pred[f_pred==test_y]==1)/sum(test_y==1)
#     print(best)
#     print(dd)
#     print(recall)
    best=0
    dd=0
    acc=0
    for delta in np.arange(0,0.9,0.01):
        mody_pred=pred_prob.copy()
        mody_pred[:,0]=mody_pred[:,0]+delta
        f_pred=np.argmax(mody_pred,1)
        if sum(f_pred==test_y)/len(f_pred)> best:
            best=sum(f_pred==test_y)/len(f_pred)
            dd=delta
            recall=sum(f_pred[f_pred==test_y]==1)/sum(test_y==1)
    print('accu ',best)
    print('delta ',dd)
    print('best recall',recall)
    print(sum(test_y))
    print(len(test_y))
    #break
# #     pred_prob=lg.predict_proba(test_x)
#     fpr, tpr, thresholds = metrics.roc_curve(test_y, pred_prob[:,1], pos_label=1)
#     auc=metrics.auc(fpr, tpr)
#     accuracy=sum(np.argmax(pred_prob,1)==test_y)/len(test_y)
    
#     print("lg auc: ",auc)
#     print("lg accuracy: ",accuracy)

--------------------------------------------------
Industry:  Basic Industries
D 0.874168207927
rf auc:  1.0
rf accuracy:  0.9921875
accu  0.9921875
delta  0.0
best recall 0.984375
64
128
--------------------------------------------------
Industry:  Consumer Services
D 0.714254651893
rf auc:  1.0
rf accuracy:  0.996598639456
accu  1.0
delta  0.01
best recall 1.0
147
294
--------------------------------------------------
Industry:  Finance
D 0.831848004927
rf auc:  1.0
rf accuracy:  0.993670886076
accu  0.993670886076
delta  0.0
best recall 0.987341772152
79
158
--------------------------------------------------
Industry:  Health Care
D 0.9348125
rf auc:  1.0
rf accuracy:  0.989583333333
accu  0.989583333333
delta  0.0
best recall 0.979166666667
48
96
--------------------------------------------------
Industry:  Miscellaneous
D 0.828399152276
rf auc:  1.0
rf accuracy:  1.0
accu  1.0
delta  0.0
best recall 1.0
94
188
--------------------------------------------------
Industry:  Public Ut

--------------------------------------------------
Industry:  Basic Industries
D 0.802533333333
rf auc:  0.6953125
rf accuracy:  0.59375
accu  0.59375
delta  0.0
best recall 0.4375
16
32
--------------------------------------------------
Industry:  Consumer Services
D 0.702235294118
rf auc:  0.691020408163
rf accuracy:  0.542857142857
accu  0.542857142857
delta  0.0
best recall 0.114285714286
35
70
--------------------------------------------------
Industry:  Finance
D 0.782727272727
rf auc:  0.756143667297
rf accuracy:  0.673913043478
accu  0.673913043478
delta  0.0
best recall 0.434782608696
23
46
--------------------------------------------------
Industry:  Health Care
D 0.781
rf auc:  0.777777777778
rf accuracy:  0.5
accu  0.5
delta  0.0
best recall 0.0
3
6
--------------------------------------------------
Industry:  Media
D 0.695
rf auc:  0.777777777778
rf accuracy:  0.5
accu  0.5
delta  0.0
best recall 0.0
3
6
--------------------------------------------------
Industry:  Miscell

In [80]:
clf.predict_proba(train_x[train_y==1])

array([[ 0.31658633,  0.68341367],
       [ 0.33471296,  0.66528704],
       [ 0.32849316,  0.67150684],
       ..., 
       [ 0.0033835 ,  0.9966165 ],
       [ 0.00184607,  0.99815393],
       [ 0.00687946,  0.99312054]])

In [57]:
probs=pd.DataFrame({'pred':pred_prob[:,0],'pred_1':pred_prob[:,1],'target':test_y})

In [62]:
max(probs[probs.target==0].pred_1)

0.73013097099873081

In [61]:
probs

,pred,pred_1,target
0,0.016598,0.983402,1
1,0.008119,0.991881,1
2,0.007556,0.992444,1
3,0.003665,0.996335,1
4,0.003241,0.996759,1
5,0.026824,0.973176,1
6,0.009022,0.990978,1
7,0.002869,0.997131,1
8,0.008905,0.991095,1
9,0.012693,0.987307,1
